In [ ]:
# import dependencies

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np


# may delete or move these these once model is more defined
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [ ]:
# import data and create dataframe

In [ ]:
# partition data into training and test data


In [ ]:
# create ml model

In [ ]:
# train the model

In [ ]:
# make preditions with the model

In [ ]:
# validate the model